<a id="1"></a>
# <div style="padding: 20px; color: cyan; margin: 0; font-size: 40px; font-family: Arial; text-align: left; border-radius: 5px; background-color: #000000; border: 2px solid blue;"><b>Preprocessing</b></div>

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 30px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Import</b>

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_module import make_dataset
print("Setup Complete!")

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 30px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Exploring</b> data

> <span style='font-size:25px; font-family:Arial; color: #87CEFA;'><b>
Read raw data </b></span>

In [ ]:
def is_path_exists(path):
    if not os.path.exists(path):
        return False
    return True

In [ ]:
path = '../data/raw/raw_data.csv'
if is_path_exists(path):
    raw_df = pd.read_csv(path)
else:
    print("File not found!")

> <span style='font-size:25px; font-family:Arial; color: #87CEFA;'><b>
Number of rows and columns in the crawled data </b></span>

In [ ]:
# Data columns
print(f"Data shape: {raw_df.shape}")
print(f"Rows = {raw_df.shape[0]} | Columns = {raw_df.shape[1]}")

In [ ]:
# Data columns
raw_df.head(2)

> <span style='font-size:25px; font-family:Arial; color: #87CEFA;'><b>
De-duplicate for the raw data </b></span>

In [ ]:
def is_duplicated_rows(df):
    index = df.index
    detect_dup_series = index.duplicated(keep='first')
    num_duplicated_rows = detect_dup_series.sum()
    return True if num_duplicated_rows > 0 else False

In [ ]:
if is_duplicated_rows(raw_df):
    raw_df = raw_df.drop_duplicates(keep='first')
    print('Shape after de-deduplicate: ', raw_df.shape)
else:
    print('No duplicated row.')

> <span style='font-size:25px; font-family:Arial; color: #87CEFA;'><b>
Data type of each columns and comment about the suitability</b></span>

[//]: <> (Text)
<span style="font-size:20px; font-family:Arial;"> 

Data type I think should be changed:
- float64 -> int64: `min_players`, `max_players`, `min_playtime`, `max_playtime`, `playing_time`, `min_age`
- float64 -> nominal: `year`

</span>

In [ ]:
def find_data_type(df):
    cat_cols = df.select_dtypes(include=['object', 'bool']).columns.tolist()
    num_cols = df.select_dtypes(exclude=['object', 'bool']).columns.tolist()
    
    return cat_cols, num_cols

In [ ]:
cat_cols, num_cols = find_data_type(raw_df)
print(f"Categorical columns: {cat_cols}")
print(f"Numerical columns: {num_cols}")

In [ ]:
# Raw data state
dtype = raw_df.dtypes
dtype

> <span style='font-size:25px; font-family:Arial; color: #87CEFA;'><b>
Distribution of numeric, text and categorial data</b></span>

In [ ]:
# Check numeric data's status
numeric_info = raw_df.select_dtypes(exclude=['object', 'bool'])

def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

numeric_info = numeric_info.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
numeric_info

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
# warnings.simplefilter(action='ignore', category=UserWarning)
# import math

# num_rows = math.ceil(len(num_cols) / 3)
# fig, ax = plt.subplots(num_rows, 3, figsize=(20, 20))

# # Flatten the axes array and iterate over it and the columns simultaneously
# for ax, col in zip(ax.flatten(), num_cols):
#     sns.histplot(x=raw_df[col], ax=ax)
#     ax.set_title(f"{col} Distribution", fontsize=15)
#     ax.set_xlabel(col, fontsize=12)

# # Remove any unused subplots
# for ax in ax.flatten()[len(num_cols):]:
#     ax.remove()

# plt.tight_layout()
# plt.show()

In [ ]:
missing_num_cols = numeric_info.loc['missing_ratio', numeric_info.loc['missing_ratio'] > 0].index.tolist()
print(f"Missing columns: {missing_num_cols}")

In [ ]:
# visualize missing data

In [ ]:
# Check categorial data's status
category_info = raw_df.select_dtypes(include=['object', 'bool'])

def num_values(s):
    s = s.astype(str).str.split(',')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.astype(str).str.split(',')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

category_info = category_info.agg([missing_ratio, num_values, value_ratios])
category_info

In [ ]:
missing_cat_cols = category_info.loc['missing_ratio', category_info.loc['missing_ratio'] > 0].index.tolist()
print(f"Missing columns: {missing_cat_cols}")

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 30px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Cleaning </b> data

[//]: <> (Text)
<span style="font-size:20px; font-family:Arial;"> 

After understanding data, we need to clean data by:
- `remove column with above half missing value`
- `fill in those missing values`
</span>

In [ ]:
def drop_missing_features(df, missing_lst, threshold) -> pd.DataFrame:
    """Drop missing features from Pandas dataframe base on given threshold.

    Args:
        df (pd.DataFrame): Input dataframe for processing.
        missing_lst (dict, optional): Dict missing value rate which calculated from previous stage.
        threshold (float, optional): User threshold for dropping. (%)

    Returns:
        pd.DataFrame: Updated dataframe
    """
    if (df is None) or (missing_lst is None) or (threshold is None):
        print(f"[LOG] Invalid.")
        raise ValueError
    
    # Create a copy of the dataframe to avoid changing the original
    df_cp = df.copy()
    
    # List to maintain the columns to drop
    cols_to_trim = []
    
    for key, value in missing_lst.items():
        if float(value) > threshold:
            cols_to_trim.append(key)
            
    if len(cols_to_trim) > 0:
        # drop the columns identified using the dataframe drop() method
        df_cp = df_cp.drop(columns=cols_to_trim)
        print("[LOG] Dropped column(s): " + ", ".join(cols_to_trim))
    else:
        print("[LOG] Have no column(s) to trim.")
        
    return df_cp

In [ ]:
# Drop columns with missing ratio > 50%
dropped_missing_df = drop_missing_features(raw_df, dict(missing_df.iloc[0]), 50.0)
dropped_missing_df.shape

In [ ]:
def filling_missing_value(df) -> pd.DataFrame:
    if (df is None):
        print(f"[LOG] Invalid.")
        raise ValueError
    
    # Create a copy of the dataframe to avoid changing the original
    df_cp = df.copy()
    
    # Get the list of columns in the dataframe
    cols_list = list(df_cp.columns)
    
    # Replace all None values with NaN, fillna only works on nans
    df_cp.fillna(value=np.nan, inplace=True)
    
    # Replace all NaN values with the mean of the column values
    for col in cols_list:
        # categorial data
        if df_cp[col].dtype in ['object', 'bool']:
            df_cp[col].fillna((df_cp[col].mode()[0]), inplace=True)

        # numeric data
        elif df_cp[col].dtype in ['int64', 'float64']:
            df_cp[col].fillna((df_cp[col].mean()), inplace=True)
        
    return df_cp
    

In [ ]:
# Fill missing values in dataframe
filled_missing_df = filling_missing_value(dropped_missing_df)

new_missing_df = filled_missing_df.agg([missing_ratio])
new_missing_df

> <span style='font-size:25px; font-family:Arial; color: #87CEFA;'><b>
Visualizing data to find outliers </b></span>

In [ ]:
# correlation matrix
correlation_matrix = filled_missing_df.select_dtypes(exclude=['object', 'bool']).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.show()

In [ ]:
# Visualizing data
for col in filled_missing_df.select_dtypes(exclude=['object', 'bool']):
    temp_df = filled_missing_df[col]
    plt.scatter(temp_df.index, temp_df)
    plt.title(col)
    plt.xlabel('Index')
    plt.ylabel(col)
    plt.show()

### Comment about outliers:

After cleaning, there are some columns that have outliers to be eliminated: 

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 30px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Removing </b> Outliers

In [ ]:
# 'min_players', 'max_players', 'min_playtime', 'max_playtime', 'playing_time'
# remove top 0.5% highest outliers & value = 0
column_names = ['min_players', 'max_players', 'min_playtime', 'max_playtime', 'playing_time']
remove_outliers_df = filled_missing_df.copy()

# Remove top 0.5% highest outliers
threshold_value = remove_outliers_df[column_names].quantile(0.995)
outlier_mask = (remove_outliers_df[column_names] <= threshold_value).all(axis=1)
remove_outliers_df = remove_outliers_df[outlier_mask]

# Remove rows where any value in 'min_players' or 'max_players' is 0
zero_mask = (remove_outliers_df[column_names] > 0).all(axis=1)
remove_outliers_df = remove_outliers_df[zero_mask]

# Update the index
remove_outliers_df.reset_index(drop=True, inplace=True)

remove_outliers_df.shape

In [ ]:
# 'Year'

# remove year < 1900s
remove_outliers_df = remove_outliers_df[remove_outliers_df['year'] >= 1900]

# Reset the index after removing rows
remove_outliers_df.reset_index(drop=True, inplace=True)

remove_outliers_df.shape

In [ ]:
# Visualizing data
for col in remove_outliers_df.select_dtypes(exclude=['object', 'bool']):
    temp_df = remove_outliers_df[col]
    plt.scatter(temp_df.index, temp_df)
    plt.title(col)
    plt.xlabel('Index')
    plt.ylabel(col)
    plt.show()

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 30px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Transformating </b> data

[//]: <> (Text)
<span style="font-size:20px; font-family:Arial;"> 

Some column's data type is not suitable, we need to change that
</span>

In [ ]:
# Cast suitable data type for some columns
casted_type_df = remove_outliers_df.copy()

# float64 -> str
casted_type_df['year'] = casted_type_df['year'].astype(str)

# float64 -> int64
temp_name_list = ['min_players', 'max_players', 'min_playtime', 'max_playtime', 'playing_time', 'min_age']
casted_type_df[temp_name_list] = casted_type_df[temp_name_list].astype(np.int64)

# recheck
casted_type_df.info()

In [ ]:
# drop some un-used columns: 'description', 'publisher', 'family'
re_construct_df = casted_type_df.copy()
re_construct_df = re_construct_df.drop(columns=['description', 'publisher', 'family'])
re_construct_df.shape

In [ ]:
# re-construct some columns: 'category', 'mechanism'
re_construct_df['category'] = re_construct_df['category'].apply(lambda x: [category.strip() for category in x.split(',')])
re_construct_df['mechanism'] = re_construct_df['mechanism'].apply(lambda x: [mechanism.strip() for mechanism in x.split(',')])

re_construct_df.head(1)[['category', 'mechanism']]

In [ ]:
re_construct_df = re_construct_df.explode('category')
re_construct_df = re_construct_df.explode('mechanism')

In [ ]:
re_construct_df.shape

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Saving </b> data

In [ ]:
# final check
final_df = re_construct_df.copy()
final_df.info()

In [ ]:
final_df.head(2)

In [ ]:
# Save preprocessed data into csv file
preprocessed_df = final_df.copy()
preprocessed_df.to_csv(f"../data/processed/processed_data.csv", index=False)